### Importing libraries

In [115]:
import pandas as pd
import altair as alt
import numpy as np
from scipy.integrate import quad
from scipy.stats import norm
from scipy.optimize import minimize

In [116]:
alt.themes.enable('dark')
alt.data_transformers.disable_max_rows()
alt.warnings.simplefilter(action='ignore')

### Read CSV (ISC)

In [153]:
EQ_data = pd.read_csv("ANSS_catalog_1950_2009.csv") #, usecols=[0,1,2,3,4,5,6,7,8,9,10,11])
# EQ_data = EQ_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)
# EQ_data['MAG'] = EQ_data['MAG'].astype('float32')

EQ_data.rename(str.upper, axis='columns', inplace=True)

### Highlight nuclear explosions

In [154]:
# mask = ((EQ_data['TYPE'] == 'kn') & (EQ_data['MAG'].isna() != True))
# EQ_data = EQ_data[mask]
EQ_data = EQ_data.dropna(subset=['MAG'])
EQ_data = EQ_data[EQ_data['MAG'] >= 0]
# EQ_data['MAGTYPE'].value_counts()

### Specify mag type

In [155]:
EQ_data['MAGTYPE'].value_counts()

MAGTYPE
md         545223
mc         238152
ml         228236
mb         152585
mh          81428
mw          18104
mblg         5030
ms           4523
mwc          3202
ma           2821
mlg          1254
m            1172
mwb           543
mun           135
mb_lg         112
lg             70
fa             24
uk             18
mwr            15
Unknown         1
mww             1
Md              1
Name: count, dtype: int64

In [156]:
prev_EQ = EQ_data

In [157]:
def getYear(time):
    return int(str.split(time, '-')[0])

EQ_data['YEAR'] = EQ_data['TIME'].apply(getYear)

In [158]:
mag_year_data = EQ_data.groupby('YEAR')['MAGTYPE'].value_counts()
selection = alt.selection_multi(fields=['MAGTYPE'], bind='legend')
hist_MY = alt.Chart(mag_year_data.reset_index()).mark_bar().add_selection(selection).encode(
    alt.X('YEAR'),
    alt.Y('count'),
    color='MAGTYPE',
    opacity=alt.condition(selection, alt.value(1), alt.value(0))
).interactive()
hist_MY

alt.Chart(...)

In [432]:
EQ_data = prev_EQ
# EQ_data = EQ_data[EQ_data['MAGTYPE'].isin(['mb', 'ms', 'mwc', 'mwb', 'uk'])]
# EQ_data = EQ_data[EQ_data['MAGTYPE'].isin(['mh', 'ml', 'md', 'mc'])]
EQ_data = EQ_data[EQ_data['MAGTYPE'].isin(['mb', 'ms', 'mwc', 'mwb', 'uk', 'm'])]

### Plot GR graph

In [433]:
hist_MN = alt.Chart(EQ_data['MAG'].value_counts().reset_index()).mark_bar().encode(
    alt.X('MAG'),
    alt.Y('count')
).interactive()

hist_NY = alt.Chart(EQ_data['YEAR'].value_counts().reset_index()).mark_bar().encode(
    alt.X('YEAR'),
    alt.Y('count')
)

In [434]:
Y_cumulative = []
GR_source = EQ_data['MAG'].value_counts().reset_index().sort_values(by='MAG')

for m in GR_source['MAG']:
    Y_cumulative.append(np.sum(GR_source[GR_source['MAG'] >= m]['count']))

source = pd.DataFrame({
  'M': GR_source['MAG'],
  'logN': np.log10(GR_source['count']),
  'logN (cumulative)': np.log10(Y_cumulative) 
})
GR_chart = alt.Chart(source.melt('M')).mark_point(filled=True).encode(
    alt.X('M', scale=alt.Scale(domain=(0,9))),
    y='value',
    color='variable',
).interactive()

hist_MN | GR_chart | hist_NY

alt.HConcatChart(...)

## Estimate magnitude of completeness

### OK1993 method

In [435]:
def q(m, mu, sig):
    return quad(norm.pdf, -np.inf, m, args=(mu, sig))[0]

def f(m, b, mu, sig):
    return b*np.exp((-b)*(m-mu) - (b*sig)**2/2) * q(m, mu, sig)

def neg_logL(parameters):
    b, mu, sig = parameters
    res = np.sum([np.log(f(m, b, mu, sig))*count for m, count in EQ_data['MAG'].value_counts().items()])
    return -res

In [436]:
# neg_logL([1,5,1])

In [437]:
# mle_model = minimize(neg_logL, np.array([1,0,1]), method='BFGS')
# mle_model

### b−value stability method (MBS)

In [438]:
delta_m = 0.1   # bin size
dM = 0.5        # in b_ave

In [439]:
def frange(x, y, jump):
    while x < y:
        yield x
        x += jump

def preCalc(Data):

    b_list = {}
    db_list = {}

    for M_c in frange(0.0, 10.0, delta_m):
        M_c = round(M_c, 2)
        Sample = Data[Data['MAG'] >= M_c]['MAG']
        M_mean = Sample.mean()
        N = Sample.size

        b = np.log10(np.e)/(M_mean - M_c + delta_m/2)
        db = 2.3*b**2*np.sqrt(np.sum((Sample - M_mean)**2)/N/(N-1))

        b_list[M_c] = b
        db_list[M_c] = db
    
    return b_list, db_list

In [440]:
def calcMc(b_list, db_list):

    for M_c in frange(0.0, 9.0, delta_m):
        M_c = round(M_c, 2)
        b = b_list[M_c]
        db = db_list[M_c]

        num = 0
        b_ave = 0
        for M_co in frange(M_c, M_c+dM, delta_m):
            M_co = round(M_co, 2)
            b_ave += b_list[M_co]
            num += 1
        b_ave /= num    

        if abs(b_ave - b) <= db:
            break
    
    return M_c, b

In [441]:
b_list, db_list = preCalc(EQ_data)
M_c, b = calcMc(b_list, db_list)

M_c, b

(5.7, np.float64(1.075840887448942))

In [442]:
a = np.log10(EQ_data[abs(EQ_data['MAG'] - M_c) < delta_m/2].shape[0])

In [449]:
x = np.linspace(M_c-1, 8, 100)
source = pd.DataFrame({'x': x, 'f(x)': a - b*(x-M_c)})

GR_line = alt.Chart(source).mark_line(
    color='cornsilk',
    strokeWidth=1.5
).encode(
    x='x',
    y='f(x)'
)

rule = alt.Chart(pd.DataFrame({
  'M': [M_c],
})).mark_rule(
    color="darksalmon",
    strokeWidth=1.5,
    strokeCap='round',
    strokeDash=[8,8],
).encode(x='M')

In [450]:
hist_MN | GR_line + (GR_chart+rule) | hist_NY

alt.HConcatChart(...)

### Cutoff magnitude evolution over time

In [445]:
def getMc(year):
    b_list, db_list = preCalc(EQ_data[EQ_data['YEAR'] <= year])
    M_c, b = calcMc(b_list, db_list)
    return M_c, b

In [446]:
yMc_data = pd.DataFrame({'year': EQ_data['YEAR'].unique()})

bMc_data = []
for y in yMc_data['year']:
    bMc_data.append(getMc(y))


In [447]:
yMc_data['M_c'] = np.array(bMc_data, ndmin=2)[:,0]
yMc_data['b'] = np.array(bMc_data, ndmin=2)[:,1]

In [448]:
Mc_chart = alt.Chart(yMc_data).mark_line(
    color='cornsilk',
    strokeWidth=1.5
).encode(
    x='year', 
    y=alt.Y('M_c', )#scale=alt.Scale(domain=(4,7)))
)

b_chart = alt.Chart(yMc_data).mark_line(
    color='lightblue',
    strokeWidth=1.5
).encode(
    x='year', 
    y=alt.Y('b', )#scale=alt.Scale(domain=(0.5,1.5)))
).interactive()

Mc_chart | b_chart

alt.HConcatChart(...)